In [1]:
import pandas as pd
import numpy as np
from src.data.import_data import import_data
from src.data.clean_data import data_cleansing
from src.features.build_features import feature_generation
from src.data.make_dataset import get_sample

#### Load data into Dataframe

In [2]:
# import data
df=import_data()

In [3]:
#df_sample=get_sample(df,300000)

#### select 300.000 entries and denote them with 1 for right translation, then take the next 300.000 entries and shift the lines, so that we have negative examples, afterwards denote them by one

In [4]:
#df_sample

NameError: name 'df_sample' is not defined

#### Data Cleansing: Tokenize by white spaces, lowercase, stopword removal

In [3]:
### get first 10.000 entries to test and get features
df_selected = df.sort_index().loc[0:10000]

In [4]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)
df_selected.head(100)

,English,German
0,Resumption of the session,Wiederaufnahme der Sitzungsperiode
1,"I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant fest...","Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie..."
2,"Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.","Wie Sie feststellen konnten, ist der gefürchtete ""Millenium-Bug "" nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden."
3,"You have requested a debate on this subject in the course of the next few days, during this part-session.",Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen.
4,"In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the vari...","Heute möchte ich Sie bitten - das ist auch der Wunsch einiger Kolleginnen und Kollegen -, allen Opfern der Stürme, insbesondere in den verschiedenen Ländern der Europäischen Union, in einer Schwei..."
...,...,...
95,There was a vote on this matter.,Es gab eine Abstimmung zu diesem Punkt.
96,"As I recall, the outcome of this vote was 422 votes to 180 with a few abstentions.",Diese Abstimmung ist meiner Erinnerung nach so ausgegangen: 422 gegen 180 Stimmen bei einigen wenigen Enthaltungen.
97,"This means that all the Groups with the exception of the non-attached Members - but, of course, they are not a Group - were in agreement; only your Group thought that we should proceed as you have...","Das heißt, alle Fraktionen, mit Ausnahme der Fraktionslosen - aber die sind ja keine Fraktion - waren sich einig, nur Ihre Fraktion war der Meinung, so zu verfahren, wie Sie es hier vorgeschlagen ..."
98,All of the others were of a different opinion.,Alle anderen waren anderer Meinung.


In [5]:
df_sel=data_cleansing(df_selected)

In [6]:
df_sel

,English,German,English_orig,German_orig,SWords_dif,SWords_dif_rel,SWords_dif_normal
0,"[resumption, session]","[wiederaufnahme, sitzungsperiode]",Resumption of the session,Wiederaufnahme der Sitzungsperiode,-1,-0.500000,-0.166667
1,"[declare, resumed, session, european, parliament, adjourned, friday, 17, december, 1999, ,, would, like, wish, happy, new, year, hope, enjoyed, pleasant, festive, period, .]","[erkläre, freitag, ,, 17., dezember, unterbrochene, sitzungsperiode, europäischen, parlaments, wiederaufgenommen, ,, wünsche, nochmals, gute, jahreswechsel, hoffe, ,, schöne, ferien, .]","I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant fest...","Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie...",-4,-0.235294,-0.031061
2,"[although, ,, seen, ,, dreaded, 'millennium, bug, ', failed, materialise, ,, still, people, number, countries, suffered, series, natural, disasters, truly, dreadful, .]","[feststellen, konnten, ,, gefürchtete, ``, millenium-bug, ``, eingetreten, ., bürger, unserer, mitgliedstaaten, opfer, schrecklichen, naturkatastrophen, geworden, .]","Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.","Wie Sie feststellen konnten, ist der gefürchtete ""Millenium-Bug "" nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.",-5,-0.357143,-0.042735
3,"[requested, debate, subject, course, next, days, ,, part-session, .]","[parlament, besteht, wunsch, aussprache, verlauf, sitzungsperiode, nächsten, tagen, .]","You have requested a debate on this subject in the course of the next few days, during this part-session.",Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen.,-4,-0.333333,-0.100840
4,"[meantime, ,, like, observe, minute, ', silence, ,, number, members, requested, ,, behalf, victims, concerned, ,, particularly, terrible, storms, ,, various, countries, european, union, .]","[heute, möchte, bitten, -, wunsch, kolleginnen, kollegen, -, ,, opfern, stürme, ,, insbesondere, verschiedenen, ländern, europäischen, union, ,, schweigeminute, gedenken, .]","In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the vari...","Heute möchte ich Sie bitten - das ist auch der Wunsch einiger Kolleginnen und Kollegen -, allen Opfern der Stürme, insbesondere in den verschiedenen Ländern der Europäischen Union, in einer Schwei...",-6,-0.272727,-0.035653
...,...,...,...,...,...,...,...
9996,"[example, ,, accept, new, roads, accept, new, pollution, within, european, union, ., goes, totally, policies, proposing, fight, greenhouse, gas, emissions, ,, example, .]","[beispiel, neue, straßen, akzeptiert, ,, akzeptiert, neue, immissionen, europäischen, union, ,, widerspricht, völlig, politik, ,, z, ., b., bekämpfung, emission, treibhausgasen, vorschlagen, .]","For example, to accept new roads is to accept new pollution within the European Union. This goes totally against the policies which we are proposing to fight greenhouse gas emissions, for example.","Wenn man zum Beispiel neue Straßen akzeptiert, dann akzeptiert man neue Immissionen in der Europäischen Union, und dies widerspricht völlig der Politik, die wir z. B. zur Bekämpfung der Emission v...",2,0.153846,0.033626
9997,"[would, also, ask, commission, ensure, environmental, pillar, eu, ', policy, integrated, projects, submitted, goal, reducing, co2, example, forms, experimental, added, value, projects, proposed, 

#### Feature Generation

In [8]:
df_selected=feature_generation(df_sel)

In [9]:
df_selected

,English,German,English_orig,German_orig,SWords_dif,SWords_dif_rel,SWords_dif_normal,PM_dif,PM_dif_rel,PM_dif_normal,Words_dif,Words_dif_rel,Words_dif_normal,Words_unique_dif,Words_unique_dif_rel,Words_unique_dif_normal,dif_!,"dif_""",dif_#,dif_$,dif_%,dif_&,dif_',dif_(,dif_),dif_*,dif_+,"dif_,",dif_-,dif_/,dif_:,dif_;,dif_<,dif_=,dif_>,dif_?,dif_@,dif_[,dif_\,dif_],dif_^,dif__,dif_`,dif_{,dif_|,dif_},dif_~,dif_...,char_dif,char_dif_rel,char_dif_normal,char_dif_avg,English_POS,German_POS,dif_ADJ,dif_ADP,dif_ADV,dif_AUX,dif_CONJ,dif_CCONJ,dif_DET,dif_INTJ,dif_NOUN,dif_NUM,dif_PRT,dif_PRON,dif_PROPN,dif_SCONJ,dif_SYM,dif_VERB,dif_X,English_times,German_times,Pres_dif,Past_dif,other_tense_dif
0,"[resumption, session]","[wiederaufnahme, sitzungsperiode]",Resumption of the session,Wiederaufnahme der Sitzungsperiode,-1,-0.500000,-0.166667,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0.480000,6.000000,6.000000,"[(Resumption, NOUN), (of, ADP), (the, DET), (session, NOUN)]","[(Wiederaufnahme, NOUN), (der, DET), (Sitzungsperiode, NOUN)]",0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[],0,0,0
1,"[declare, resumed, session, european, parliament, adjourned, friday, 17, december, 1999, ,, would, like, wish, happy, new, year, hope, enjoyed, pleasant, festive, period, .]","[erkläre, freitag, ,, 17., dezember, unterbrochene, sitzungsperiode, europäischen, parlaments, wiederaufgenommen, ,, wünsche, nochmals, gute, jahreswechsel, hoffe, ,, schöne, ferien, .]","I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant fest...","Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie...",-4,-0.235294,-0.031061,2,2.000000,0.106522,-5,-0.238095,-0.113043,-5,-0.238095,-0.113043,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-4,-0.019139,1.163043,2.860119,"[(I, PRON), (declare, VERB), (resumed, VERB), (the, DET), (session, NOUN), (of, ADP), (the, DET), (European, PROPN), (Parliament, PROPN), (adjourned, VERB), (on, ADP), (Friday, PROPN), (17, NUM), ...","[(Ich, PRON), (erkläre, VERB), (die, DET), (am, ADP), (Freitag, NOUN), (,, PUNCT), (dem, DET), (17., ADJ), (Dezember, NOUN), (unterbrochene, ADJ), (Sitzungsperiode, NOUN), (des, DET), (Europäische...",0,0,0,-1,0,0,-1,0,3,-2,0,-1,-4,0,0,-2,0,"[(declare, Pres), (resumed, Past), (adjourned, Past), (like, ), (wish, ), (enjoyed, Past)]","[(erkläre, Pres), (wünsche, Pres), (hoffe, Pres), (hatten, Past)]",-2,2,2
2,"[although, ,, seen, ,, dreaded, 'millennium, bug, ', failed, materialise, ,, still, people, number, countries, suffered, series, natural, disasters, truly, dreadful, .]","[feststellen, konnten, ,, gefürchtete, ``, millenium-bug, ``, eingetreten, ., bürger, unserer, mitgliedstaaten, opfer, schrecklichen, naturkatastrophen, geworden, .]","Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.","Wie Sie feststellen konnten, ist der gefürchtete ""Millenium-Bug "" nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.",-5,-0.357143,-0.042735,-3,-0.750000,-0.122995,-3,-0.176471,0.050802,-4,-0.235294,-0.008021,0,0,0,0,0,0,-1,0,0,0,0,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3,-0.016043,2.323529,2.142857,"[(Although, SCONJ), (,, PUNCT), (as, ADP), (you, PRON), (will, AUX), (have, AUX), (seen, VERB), (,, PUNCT), (the, DET), (dreaded, VERB), (', PUNCT), (millennium, NOUN), (bug, NOUN), (', PUNCT), (f...","[(Wie, SCONJ), (Sie, PRON), (feststellen, VERB), (konnten, AUX), (,, PUNCT), (ist, AUX), (der, DET), (gefürchtete, ADJ), ("", PUNCT), (Millenium-

### Add direct translation of German to English and English to German

Die Übersetzung ist ein krasser Zeitaufwand und daher vorerst mit diesen Package nicht feasible.

In [23]:
# import data
df = import_data()
df_selected = df.sort_index().loc[0:10000]

In [ ]:
import googletrans
from googletrans import Translator
translator = Translator()

In [ ]:
# df_selected["English_to_German"] = df_selected["English"].apply(lambda text: translator.translate(text, src="en", dest='de').text)

In [ ]:
# df_selected["German_to_English"] = df_selected["English"].apply(lambda text: translator.translate(text, src="en", dest='de').text)

### Add segment analysis for german and english

### Add Named Entity Recognition

In [17]:
import spacy
nlp_en = spacy.load("en_core_web_sm")
nlp_de = spacy.load("de_core_news_sm")

In [18]:
def get_ner_text_en(text):
    doc = nlp_en(text)
    test = []
    for ent in doc.ents:
        test.append([ent.text,  ent.label_])
    return test

In [19]:
def get_ner_text_de(text):
    doc = nlp_de(text)
    test = []
    for ent in doc.ents:
        test.append([ent.text,  ent.label_])
    return test

In [22]:
df_selected["English_NER"] = df_selected.English_orig.apply(lambda x: get_ner_text_en(x))
df_selected["German_NER"] = df_selected.German_orig.apply(lambda x: get_ner_text_de(x))

#### Müssen uns überlegen, wie wir die Entities vergleichen wollen, weil in Englisch deutlich, deutlich mehr gelabelt wird(>10) als in Deutsch (4)

In [23]:
df_selected

,English,German,English_orig,German_orig,SWords_eng,SWords_ger,SWords_dif,SWords_dif_rel,SWords_dif_normal,Polarity_difference,Subjectivity_difference,English_NER,German_NER
0,"[resumption, of, the, session]","[wiederaufnahme, der, sitzungsperiode]",Resumption of the session,Wiederaufnahme der Sitzungsperiode,2,1,-1,-0.500000,-0.166667,0.000000,0.000000,[],[]
1,"[i, declare, resumed, the, session, of, the, european, parliament, adjourned, on, friday, 17, december, 1999, ,, and, i, would, like, once, again, to, wish, you, a, happy, new, year, in, the, hope...","[ich, erkläre, die, am, freitag, ,, dem, 17., dezember, unterbrochene, sitzungsperiode, des, europäischen, parlaments, für, wiederaufgenommen, ,, wünsche, ihnen, nochmals, alles, gute, zum, jahres...","I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant fest...","Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie...",17,13,-4,-0.235294,-0.031061,0.416061,-0.624242,"[[the European Parliament, ORG], [Friday 17 December 1999, DATE]]","[[Europäischen Parlaments, ORG]]"
2,"[although, ,, as, you, will, have, seen, ,, the, dreaded, 'millennium, bug, ', failed, to, materialise, ,, still, the, people, in, a, number, of, countries, suffered, a, series, of, natural, disas...","[wie, sie, feststellen, konnten, ,, ist, der, gefürchtete, ``, millenium-bug, ``, nicht, eingetreten, ., doch, sind, bürger, einiger, unserer, mitgliedstaaten, opfer, von, schrecklichen, naturkata...","Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.","Wie Sie feststellen konnten, ist der gefürchtete ""Millenium-Bug "" nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.",14,9,-5,-0.357143,-0.042735,-0.033333,-0.566667,[],"[[Millenium-Bug, PER]]"
3,"[you, have, requested, a, debate, on, this, subject, in, the, course, of, the, next, few, days, ,, during, this, part-session, .]","[im, parlament, besteht, der, wunsch, nach, einer, aussprache, im, verlauf, dieser, sitzungsperiode, in, den, nächsten, tagen, .]","You have requested a debate on this subject in the course of the next few days, during this part-session.",Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen.,12,8,-4,-0.333333,-0.100840,0.122222,-0.144444,"[[the next few days, DATE]]","[[Parlament, ORG]]"
4,"[in, the, meantime, ,, i, should, like, to, observe, a, minute, ', s, silence, ,, as, a, number, of, members, have, requested, ,, on, behalf, of, all, the, victims, concerned, ,, particularly, tho...","[heute, möchte, ich, sie, bitten, -, das, ist, auch, der, wunsch, einiger, kolleginnen, und, kollegen, -, ,, allen, opfern, der, stürme, ,, insbesondere, in, den, verschiedenen, ländern, der, euro...","In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the vari...","Heute möchte ich Sie bitten - das ist auch der Wunsch einiger Kolleginnen und Kollegen -, allen Opfern der Stürme, insbesondere in den verschiedenen Ländern der Europäischen Union, in einer Schwei...",22,16,-6,-0.272727,-0.035653,0.541667,-0.458333,"[[the European Union, ORG]]","[[Europäischen Union, ORG]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,"[for, example, ,, to, accept, new, roads, is, to, accept, new, pollution, within, the, european, union, ., this, goes, totally, against, the, policies, which, we, are, proposing, to, fight, greenh...","[wenn, man, zum, beispiel, neue, straßen, akze